In [17]:
pip install gurobipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\lifen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from gurobipy import GRB
import gurobipy as gb

In [1]:
# Define given data
months = ['May', 'June', 'July'] # No need to list 'August'
revenues = [180000, 260000, 420000, 580000]
expenses = [300000, 400000, 350000, 200000]
min_cash_balance = [25000, 20000, 35000, 18000]
max_borrow_amount = [250000, 150000, 350000]
init_cash_balance = 140000
interest_rates = {
    'May': [0.0175, 0.0225, 0.0275],
    'June': [0.0175, 0.0225],
    'July': [0.0175]
}

In [7]:
def find_model_optimization(model):
    # Decision variables
    borrow = {(i, j): model.addVar(vtype=GRB.CONTINUOUS, name=f"borrow_{i}_{j}")
          for i in months for j in range(len(interest_rates[i]))}

    # Objective function: minimize total repayment
    model.setObjective(gb.quicksum(borrow[i, j] * interest_rates[i][j] for i in months for j in range(len(interest_rates[i]))),
                   GRB.MINIMIZE)

    # Constrints
    for i in range(len(months)):
        # cash balance constraints: borrow + init_balance + revenue - expenses >= min_cash_balance
        model.addConstr(gb.quicksum(borrow[months[i], j] for j in range(len(interest_rates[months[i]]))) +
                    init_cash_balance + revenues[i] - expenses[i] >= min_cash_balance[i], f"cash_balance_{months[i]}")

        # total amount borrowed constraint
        model.addConstr(gb.quicksum(borrow[months[i], j] for j in range(len(interest_rates[months[i]]))) <=
                    max_borrow_amount[i], f"total_borrowed_{months[i]}")


        # cash balance constraint for end of July
        may_cash = gb.quicksum(borrow[months[0], j] * (1 + interest_rates[months[0]][j]) for j in range(len(interest_rates[months[0]])))
        june_cash = gb.quicksum(borrow[months[1], j] * (1 + interest_rates[months[1]][j]) for j in range(len(interest_rates[months[1]])))
        july_cash = borrow[months[2], 0] * (1 + interest_rates[months[2]][0])

        model.addConstr(may_cash + june_cash + july_cash >= 0.65 *
                (init_cash_balance + revenues[0] + init_cash_balance + revenues[1]),
                "end_of_July_constraint")

    # optimize the Model
    model.optimize()

    # display the results
    if model.status == gb.GRB.OPTIMAL:
        print("Optimal solution found!")
        total_repayment = model.objVal
        print(f"Total repayment: ${total_repayment:.2f}")

        for i in months:
            for j in  range(len(interest_rates[i])):
                print(f"Borrow in {i} at rate {interest_rates[i][j]}: ${borrow[i, j].x:.2f}")
    else:
        print("No optimal solution found.")


# Create a model
model = gb.Model('bank_repay_optimization')
find_model_optimization(model)


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 9 rows, 6 columns and 30 nonzeros
Model fingerprint: 0x24866750
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 3e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 5e+05]
Presolve removed 6 rows and 0 columns
Presolve time: 0.02s
Presolved: 3 rows, 8 columns, 13 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   6.162500e+04   0.000000e+00      0s
       3    8.0491400e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.03 seconds (0.00 work units)
Optimal objective  8.049140049e+03
Optimal solution found!
Total repayment: $8049.14
Bor